In [6]:
import gzip
import json
import codecs

from multiprocessing.dummy import Pool, Queue
from bs4 import BeautifulSoup
from lxml import etree,html as lhtml
import re
import requests
import functools

In [10]:
url = 'https://gg.deals/games/?sort=metascore&type=1'

In [11]:
def get_page(url, attempts=5, sleep_time=0.01, **kwargs):
    
    for _ in range(attempts):
        
        page = requests.get(url,**kwargs)
        if page.ok:
            return page
        
        time.sleep(sleep_time)
        
    print(url, page.status_code)

In [135]:
def get_urls_in_page(pages_text):
    soup = BeautifulSoup(pages_text, 'html.parser')
    l = []
    for link in soup.find_all('a', class_="ellipsis title"):
        l.append(link['href'])
    return l

In [136]:
def next_page(page_id):
    pages = requests.get('https://gg.deals/games/?sort=metascore&type=1&page={}'.format(page_id))
    return pages

In [385]:
def tags_genres_feautures(s):
    tags = soup.find('div', id='game-info-genres')
    tags = tags.find_all('a', class_="badge")
    local_list = []
    for i in tags:
        local_list.append(i.text)
        print(i.text)
    return local_list

In [460]:
def game_info(url):
    url_html = requests.get('https://gg.deals' + url).text
    soup = BeautifulSoup(url_html, 'lxml')
    res = {}
    res['url'] = 'https://gg.deals' + url
    res['name'] = soup.find('a', href=url).find('span').text
    
    image = soup.find('div', class_=lambda s: s and s.startswith('game-info-image')).find('img')
    image_url = image['src']
    res['image'] = image_url
    
    market_url = soup.find('a', class_="game-link-widget")
    res['market_url'] = market_url['href']
    
    counter = soup.find('div', class_=lambda s: s and s.startswith('game-collection-actions'))
    counter = counter.find_all('span', class_='count')
    cnt_list = re.findall(r'>\d+<',str(counter))
    for i in range(0,6):
        cnt_list[i] = cnt_list[i][1:-1]
    cnt_list = cnt_list[0:-1:2]# wish,alert,own
    wish_cnt = int(cnt_list[0])
    alert_cnt = int(cnt_list[1])
    own_cnt = int(cnt_list[2])
    res['wishlist_count'] = wish_cnt
    res['alert_count'] =  alert_cnt
    res['owners_count'] = own_cnt

    p = soup.find('div', class_=lambda s: s and s.startswith('game-info-details-section game-info-details-section-release'))
    p = p.find('p', class_='game-info-details-content')
    res['release_date'] = p.text
    p = soup.find('div', class_=lambda s: s and s.startswith('game-info-details-section game-info-details-section-developer'))
    p = p.find('p', class_='game-info-details-content')
    res['developer'] = p.text
    
    m_score = soup.find('a', class_=lambda s: s and s.startswith('score-circle score-metascore'))
    m_score = m_score.find('span', class_='overlay')
    res['metacritic_score'] = m_score.text
    user_score = soup.find('a', class_=lambda s: s and s.startswith('score-circle score-userscore'))
    user_score = user_score.find('span', class_='overlay')
    res['user_score'] = user_score.text
    
    tags = soup.find('div', id="game-info-tags")
    tags = tags.find_all('a', class_="badge")
    local_list = []
    for i in tags:
        local_list.append(i.text)
    res['tags'] = local_list

    genres = soup.find('div', id='game-info-genres')
    genres = genres.find_all('a', class_="badge")
    local_list = []
    for i in genres:
        local_list.append(i.text)
    res['genres'] = local_list
    
    feature = soup.find('div', id='game-info-features')
    feature = feature.find_all('a', class_="badge")
    local_list = []
    for i in feature:
        local_list.append(i.text)
    res['features'] = local_list
    
    os = soup.find('div', class_='system-requirements-section')
    os = os.find('ul', class_="bb_ul")
    res['pc_systems'] = os.find('li').text
    
    dlcs = soup.find('div', class_='games-box games-box-related-list load-more-content')
    dlcs = dlcs.find_all('a', class_="full-link")
    local_list = []
    for i in dlcs:
        local_list.append(i['href'])
    res['dlcs'] = local_list
    
#     res['review_label'] = 
#     res['review_positive_pctg'] = 
#     res['review_count'] = 
#     res['price_history'] = 
    return res

In [461]:
pages = next_page(2)
my_list= []
my_list = get_urls_in_page(pages.text)
game_info(my_list[3])
# for i in range(2):
#     game_info(my_list[i])

{'url': 'https://gg.deals/game/grand-theft-auto-san-andreas/',
 'name': 'Grand Theft Auto: San Andreas',
 'image': 'https://img.gg.deals/e3/83/126f646277b36c221907f8d49ba2d0b0bb15_307xt176.jpg',
 'market_url': 'https://gg.deals/redirect/a0abab69ee204dcdb5e7fe2a0fe49e2936df07ba/?utm_source=games%2Fsingle',
 'wishlist_count': 931,
 'alert_count': 100,
 'owners_count': 10701,
 'release_date': '06 Jun 2005',
 'developer': 'Rockstar Games',
 'metacritic_score': '93',
 'user_score': '8.9',
 'tags': ['Open World',
  'Action',
  'Crime',
  'Classic',
  'Automobile Sim',
  'Third Person',
  'Multiplayer',
  'Singleplayer',
  'Shooter',
  'Driving',
  'Story Rich',
  'Adventure',
  'Sandbox',
  'Mature',
  'Great Soundtrack',
  'Third-Person Shooter',
  'Atmospheric',
  'Moddable',
  'Funny',
  'Comedy'],
 'genres': ['Action'],
 'features': ['Single-player'],
 'dlcs': ['/ru/pack/grand-theft-auto-complete-pack/',
  '/ru/pack/grand-theft-auto-trilogy/',
  '/ru/pack/grand-theft-auto-iv-grand-theft-

In [219]:
headers = {'x-requested-with': 'XMLHttpRequest'}
test = requests.get('https://gg.deals/ru/games/chartHistoricalData/{}/?hideKeyshops=0'.format(21937), headers=headers)
tree = test.json()